Chapter 10. Advanced Expressions
===================

In [1]:
from proveit import ExprList
from proveit._common_ import x, y, z
%begin advanced_expressions
xyz = ExprList(x, y, z)

In [2]:
xy = ExprList(x, y)

xy: (x , y)

The `getElem` method may be used to access an element of the `ExprList` using an `Expression` representative for the index, relative to a `base` (defaulting to `1`, common in mathematics though it is less common in programming).

In [3]:
from proveit.number import num
xyz.getElem(num(3)) # get the 3rd element of (x, y, z)

z

The method works by starting from the `base` and adding `1` until the given index is reached while iterating through the elements of the `ExprList`.  It uses the `proveit.number` package for number representations for generating the required counting steps which are optionally passed back via the `requirements` parameter.

In [4]:
requirements = []
xyz.getElem(num(2), base=0, requirements=requirements) # start from a base of zero and pass back the requirements

z

In [5]:
ExprList(*requirements) # composing an ExprList out of the requirements list is done to display a LaTeX representation.

((0 + 1) = 1 , (1 + 1) = 2)

The *index* argument into the `getElem` method does not need to be a direct representation of a number.  If it can be proven to be equal to a counted index value (through *assumptions*, for example), that also works.

In [6]:
from proveit._common_ import k
from proveit.logic import Equals
requirements = []
xyz.getElem(k, assumptions=[Equals(k, num(2))], requirements=requirements)

y

In [7]:
ExprList(*requirements)

((1 + 1) = 2 , k = 2)

<a name="Indexed"></a>Indexed
====================

Before we discuss `Iter`s, let us first introduced `Indexed` which are designed specifically to be contained in an `Iter`.  An `Indexed` `Expression` has a *variable* (as a `Variable`) and an *index* (or *indices*).  The *variable* is intended to be a placeholder for an `ExprList` (or `ExprTensor`).  The `Indexed` object represents an element of that `ExprList` (or `ExprTensor`) addressed by the *index* (or *indices*).  Here is a simple example for a single *index* $k$ of *variable* $a$:

In [8]:
from proveit import Indexed
from proveit._common_ import a, b, k
a_k = Indexed(a, k)

a_k: a_k

In [9]:
a_k.exprInfo()

,core type,sub-expressions,expression
0,Indexed,"var: 1index: 2base: ""1""",
1,Variable,,
2,Variable,,


Note that a *base* is shown in the *expression information*.  This indicates an initial offset into the list.  The default is *base*=1 which means that 1 is taken to be the first element.  Although this *base* is not shown in the LaTeX rendering (or the string rendering), it can usually be inferred from the context of a containing `Iter`.  Let us show a *base*=0 example:

In [10]:
b_k = Indexed(b, k, base=0)

b_k: b_k

In [11]:
b_k.exprInfo()

,core type,sub-expressions,expression
0,Indexed,"var: 1index: 2base: ""0""",
1,Variable,,
2,Variable,,


The *index* may be substituted freely.

In [12]:
from proveit.logic import And
a_and_b = And(a, b)
a_k.substituted({k:a_and_b})

a_(a and b)

The *variable* may only be substituted with another `Variable` or with an `ExprList`.

In [13]:
a_k.substituted({a:b})

b_(k)

In [14]:
try:
    a_k.substituted({a:a_and_b})
    assert False, "Expecting an TypeError error; should not make it to this point"
except TypeError as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: 'var' being indexed should be a Variable


But when the *variable* is substituted with an `ExprList`, the *index* must be an `Expression` that is "known" to evaluate to a specific element (or iteration range as we see below in the `Iter` section).

In [15]:
from proveit import ExprListError
try:
    a_k.substituted({a:xyz})
    assert False, "Expecting an ExprListError error; should not make it to this point"
except ExprListError as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Could not determine the 'index'-ed element of the ExprList.


Here we assign the index, $k$, to a specific numeric value to extract the first element of the $(x, y, z)$:

In [16]:
from proveit.number import num
a_k.substituted({a:xyz, k:num(1)})

x

But we can also assign the index implicitly via *assumptions* or other known relationships:

In [17]:
from proveit.logic import Equals
from proveit.number import num
requirements=[]
a_k.substituted({a:xyz}, assumptions=[Equals(k, num(2))], requirements=requirements)

y

As with the `getElem` of the `ExprList` class, we can pass back the requirements for counting into the `ExprList` that is being substituted into the *indexed* *variable*, as attained above and displayed below.

In [18]:
ExprList(*requirements)

((1 + 1) = 2 , k = 2)

Next, we see what happens when the *index* is out-of-range.

In [19]:
try:
    a_k.substituted({a:xyz, k:num(4)})
    assert False, "Expecting an IndexError error; should not make it to this point"
except IndexError as e:
    print("EXPECTED ERROR:", e)

EXPECTED ERROR: Index, 4, past the range of the ExprList, (x , y , z)


<a name="Iter"></a>Iter
====================

An `Iter` `Expression` represents an iteration over a range.  For example,

In [20]:
from proveit import Iter
from proveit._common_ import i, j
a_it = Iter(i, Indexed(a, i), num(1), num(3))

a_it: a_(1),...,a_(3)

In [21]:
ExprList(a_it).getElem(num(2))

a_(2)

In [22]:
%end advanced_expressions

In [23]:
from proveit.number import Less, num
Less(num(3), num(4)).prove().deriveRelaxed()

ImportError: cannot import name 'relaxLessThan' from 'proveit.number.ordering._theorems_' (/home/wwitzel/Prove-It/packages/proveit/number/ordering/_theorems_.py)

In [ ]:
ExprList(a_it).getElem(num(4))

In [ ]:
ExprList(a_it, b_it).getElem(num(2))

Here, `a_it` represents a list ranging from the first element of $x$ to the third element of $x$.  If we substitute $x$ with an `ExprList`, the iteration will be expanded by successively applying `+1` (via `proveit.number` definitions of addition and `1`) to the *start index* (1 in this case) until the *end index* (3 in this case) is reached.

In [ ]:
a_it.substituted({a:xyz})

In the following case, the substituted ExprList is not long enough to cover the iteration range.

In [ ]:
try:
    a_it.substituted({a:xy})
    assert False, "Expecting an IndexError error; should not make it to this point"
except IndexError as e:
    print("EXPECTED ERROR:", e)

The `Iter` may be embedded within a containing `ExprList`.

In [ ]:
b_it = Iter(j, Indexed(b, j), num(1), k)
a13_b1k_yz = ExprList(a_it, b_it, y, z)

When expanded via *expression substition*, the result is inserted into the outer `ExprList`:

In [ ]:
a13_b1k_yz.substituted({a:xyz, b:xy, k:num(2)})

In [ ]:
a13_b1k_yz.substituted({a:xyz, b:ExprList(x, a_it, y), k:num(5)})

In [ ]:
a13_b1k_yz.substituted({a:xyz, b:ExprList(a_it), k:num(3)})

A simple `Iter` over an `Indexed` variable as above may be used as `Lambda` *parameters* or `Operation` *operators*.  For example,

In [ ]:
from proveit.number import Add
Lambda(a13_b1k_yz, Add(*a13_b1k_yz))

Note that the asterisk, `*`, above is necessary to indicate addition over the `ExprList` rather than `Add` applied to a single `ExprList` term.

An `Iter` may be applying more generally than an iteration over an **indexed** **variable**.

In [ ]:
Iter(Lambda(k, Add(k, k)), num(1), num(5))

<a name="ExprTensor"></a>ExprTensor
====================

An `ExprTensor` is the multi-dimensional analog to `ExprList`.  It has not been fully implemented, however.  This tutorial will be updated after it has been implemented.

# Next chapter: <a href="tutorial11_advanced_proofs.ipynb">Proofs Using Advanced Expressions</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>